In [1]:
import os
import requests
import json
import re
import time
import glob
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os.path import join
from slugify import slugify
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from urllib.parse import urlparse, parse_qs

base_dir = "vuelax"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    
original_file = join(base_dir, "original.csv")

In [2]:
months = {'enero':1, 'febrero':2, 'marzo':3,
          'abril':4, 'mayo':5, 'junio':6,
          'julio':7,'agosto':8, 'septiembre':9,
          'octubre':10, 'noviembre':11, 'diciembre':12}

date_regex = re.compile('(\w+) ([0-9]+), ([0-9]{4})')

def date_converter(date):
    found = date_regex.search(date)
    if found:
        return datetime.datetime(year=int(found.group(3)), month=months[found.group(1)], day=int(found.group(2)))
    else:
        return np.nan

In [3]:
data = None
last_date = None
if os.path.exists(original_file):
    data = pd.read_csv(original_file, index_col=0, parse_dates=['date'], encoding='utf-8')
    last_date = data.iloc[0]['date']
    print("Last date", last_date)
    print(data.head())

Last date 2018-01-21 00:00:00
                                     label  \
0  CDMX a India – $8,621 ¡3 nuevas fechas!   
1                    GDL a La Paz – $1,195   
2  CDMX a India – $8,621 ¡4 nuevas fechas!   
4             GDL y CDMX a China – $11,046   
6                 CUN a Ámsterdam – $9,339   

                                                 url       date  
0  http://www.vuelax.com/2018/01/21/cdmx-a-india-... 2018-01-21  
1  http://www.vuelax.com/2018/01/20/gdl-a-la-paz-... 2018-01-20  
2  http://www.vuelax.com/2018/01/20/cdmx-a-india-... 2018-01-20  
4  http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-... 2018-01-19  
6  http://www.vuelax.com/2018/01/19/cun-a-amsterd... 2018-01-19  


In [5]:
oportunidades_url = "http://www.vuelax.com/category/oportunidades/page/%d/"
uncategorized_url = "http://www.vuelax.com/category/uncategorized/page/%d/"

In [6]:
content = []
brk = False
for page in range(1, 100000):
    url = uncategorized_url % page
    op_page = requests.get(url)
    if page % 10 == 0:
        print("Requesting", url)
    if op_page.status_code != 200:
        break
    op_soup = BeautifulSoup(op_page.text, "lxml")
    main_ul = op_soup.find("ul", {"class":"penci-grid"})
    articles = main_ul.findAll("article", {"class":"item"})
    for article in articles:
        grid_title = article.find("h2", {"class":"grid-title"})
        a = grid_title.find("a")
        grid_post_box_meta = article.find("div", {"class":"grid-post-box-meta"})
        date = date_converter(grid_post_box_meta.text.strip())
        if date <= last_date:
            brk = True
            print("Ya tengo esta fecha!")
            break
        content.append([a.text, a.get('href'), date])
    if brk:
        break
        
if data is not None:
    other = pd.DataFrame(content, columns= ["label", "url", "date"])
    data = pd.concat([data, other])
else:
    data =  pd.DataFrame(content, columns= ["label", "url", "date"])
data.sort_values(by=['date', 'label'], ascending= False, inplace=True)
print(data.head())
print(data.info())

print("Done")

Ya tengo esta fecha!
                                     label  \
0  CDMX a India – $8,621 ¡3 nuevas fechas!   
1                    GDL a La Paz – $1,195   
2  CDMX a India – $8,621 ¡4 nuevas fechas!   
4             GDL y CDMX a China – $11,046   
6                 CUN a Ámsterdam – $9,339   

                                                 url       date  
0  http://www.vuelax.com/2018/01/21/cdmx-a-india-... 2018-01-21  
1  http://www.vuelax.com/2018/01/20/gdl-a-la-paz-... 2018-01-20  
2  http://www.vuelax.com/2018/01/20/cdmx-a-india-... 2018-01-20  
4  http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-... 2018-01-19  
6  http://www.vuelax.com/2018/01/19/cun-a-amsterd... 2018-01-19  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1435 entries, 0 to 1392
Data columns (total 3 columns):
label    1435 non-null object
url      1435 non-null object
date     1435 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 44.8+ KB
None
Done


In [7]:
data.to_csv(join(base_dir, "original.csv"), encoding='utf-8')

In [8]:
data = pd.read_csv(join(base_dir, "original.csv"), index_col=0, parse_dates=['date'], encoding='utf-8')

In [9]:
location_regex = re.compile('([\w0-9,\s\.]+) [a|A] ([\w0-9,\s\.]+)\s*[-|–|"desde"|"DESDE"]\s*\$([0-9\.,]+)')


clean_values = []
non_clean_values = []

for index, row in data.iterrows():
    label = row['label']
    find = location_regex.search(label)
    if find:
        de = find.group(1)
        a = find.group(2)
        por = find.group(3)
        clean_values.append([de, a, por, row["url"], row["date"]])
    else:
        non_clean_values.append(row.values)

clean = pd.DataFrame(clean_values, columns= ["origin", "destination", "price", "url", "date"])
still_dirty_df = pd.DataFrame(non_clean_values, columns= ["label", "url", "date"])



print("== Clean ==")
print(clean.head())
print(clean.info())
clean.to_csv(join(base_dir, "clean.csv"), encoding='utf-8')
print()
print("== Dirty ==")
print(still_dirty_df.head())
print(still_dirty_df.info())
still_dirty_df.to_csv(join(base_dir, "still_dirty.csv"), encoding='utf-8')

== Clean ==
       origin destination   price  \
0        CDMX      India    8,621   
1         GDL     La Paz    1,195   
2        CDMX      India    8,621   
3  GDL y CDMX      China   11,046   
4         CUN  Ámsterdam    9,339   

                                                 url       date  
0  http://www.vuelax.com/2018/01/21/cdmx-a-india-... 2018-01-21  
1  http://www.vuelax.com/2018/01/20/gdl-a-la-paz-... 2018-01-20  
2  http://www.vuelax.com/2018/01/20/cdmx-a-india-... 2018-01-20  
3  http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-... 2018-01-19  
4  http://www.vuelax.com/2018/01/19/cun-a-amsterd... 2018-01-19  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 5 columns):
origin         1297 non-null object
destination    1297 non-null object
price          1297 non-null object
url            1297 non-null object
date           1297 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 50.7+ KB
None

== Dirty 

In [10]:
still_dirty_df = pd.read_csv(join(base_dir, "still_dirty.csv"), parse_dates=['date'], index_col = 0, encoding='utf-8')
print(still_dirty_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 137
Data columns (total 3 columns):
label    138 non-null object
url      138 non-null object
date     138 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 4.3+ KB
None


In [11]:
location_regex_note = re.compile('([\w0-9,\s\.]+) [a|A] ([\w0-9,\s\.]+)\s*\(([\w\s]+)\)\s*[-|–|"desde"|"DESDE"]\s*\$([0-9\.,]+)')


clean_values = []
non_clean_values = []

for index, row in still_dirty_df.iterrows():
    label = row['label']
    find = location_regex_note.search(label)
    if find:
        de = find.group(1)
        a = find.group(2)
        note = find.group(3)
        por = find.group(4)
        clean_values.append([de, a, por, note, row["url"], row["date"]])
    else:
        non_clean_values.append(row.values)


clean2 = pd.DataFrame(clean_values, columns= ["origin", "destination", "price", "note", "url", "date"])
print(clean2.info())

still_dirty_df = pd.DataFrame(non_clean_values, columns= ["label", "url", "date"])
print(still_dirty_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
origin         11 non-null object
destination    11 non-null object
price          11 non-null object
note           11 non-null object
url            11 non-null object
date           11 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 608.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 3 columns):
label    127 non-null object
url      127 non-null object
date     127 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.1+ KB
None


In [12]:
clean = pd.read_csv(join(base_dir, "clean.csv"), index_col = 0, parse_dates=['date'], encoding='utf-8')
clean = pd.concat([clean, clean2])

print("== Clean ==")
print(clean.head())
print(clean.info())
clean.to_csv(join(base_dir, "clean.csv"), encoding='utf-8')
print()
print("== Dirty ==")
print(still_dirty_df.head())
print(still_dirty_df.info())
still_dirty_df.to_csv(join(base_dir, "still_dirty.csv"), encoding='utf-8')

== Clean ==
        date destination note      origin   price  \
0 2018-01-21      India   NaN        CDMX   8,621   
1 2018-01-20     La Paz   NaN         GDL   1,195   
2 2018-01-20      India   NaN        CDMX   8,621   
3 2018-01-19      China   NaN  GDL y CDMX  11,046   
4 2018-01-19  Ámsterdam   NaN         CUN   9,339   

                                                 url  
0  http://www.vuelax.com/2018/01/21/cdmx-a-india-...  
1  http://www.vuelax.com/2018/01/20/gdl-a-la-paz-...  
2  http://www.vuelax.com/2018/01/20/cdmx-a-india-...  
3  http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-...  
4  http://www.vuelax.com/2018/01/19/cun-a-amsterd...  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1308 entries, 0 to 10
Data columns (total 6 columns):
date           1308 non-null datetime64[ns]
destination    1308 non-null object
note           11 non-null object
origin         1308 non-null object
price          1308 non-null object
url            1308 non-null object
dtypes: datet

In [13]:
still_dirty_df = pd.read_csv(join(base_dir, "still_dirty.csv"), index_col = 0, encoding='utf-8')
print(still_dirty_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 126
Data columns (total 3 columns):
label    127 non-null object
url      127 non-null object
date     127 non-null object
dtypes: object(3)
memory usage: 4.0+ KB
None


In [14]:
location_regex_note = re.compile('¡([\w0-9,\s\.]+) [a|A] ([\w0-9,\s\.]+)!\s[-|–|"desde"|"DESDE"]\s*\$([0-9\.,]+)')


clean_values = []
non_clean_values = []

for index, row in still_dirty_df.iterrows():
    label = row['label']
    find = location_regex_note.search(label)
    if find:
        de = find.group(1)
        a = find.group(2)
        por = find.group(3)
        clean_values.append([de, a, por, row["url"], row["date"]])
    else:
        non_clean_values.append(row.values)


clean2 = pd.DataFrame(clean_values, columns= ["origin", "destination", "price", "url", "date"])
print(clean2.info())

still_dirty_df = pd.DataFrame(non_clean_values, columns= ["label", "url", "date"])
print(still_dirty_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
origin         42 non-null object
destination    42 non-null object
price          42 non-null object
url            42 non-null object
date           42 non-null object
dtypes: object(5)
memory usage: 1.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 3 columns):
label    85 non-null object
url      85 non-null object
date     85 non-null object
dtypes: object(3)
memory usage: 2.1+ KB
None


In [15]:
clean = pd.read_csv(join(base_dir, "clean.csv"), index_col = 0, encoding='utf-8')
clean = pd.concat([clean, clean2])

print("== Clean ==")
print(clean.head())
print(clean.info())
clean.to_csv(join(base_dir, "clean.csv"), encoding='utf-8')
print()
print("== Dirty ==")
print(still_dirty_df.head())
print(still_dirty_df.info())
still_dirty_df.to_csv(join(base_dir, "still_dirty.csv"), encoding='utf-8')

== Clean ==
         date destination note      origin   price  \
0  2018-01-21      India   NaN        CDMX   8,621   
1  2018-01-20     La Paz   NaN         GDL   1,195   
2  2018-01-20      India   NaN        CDMX   8,621   
3  2018-01-19      China   NaN  GDL y CDMX  11,046   
4  2018-01-19  Ámsterdam   NaN         CUN   9,339   

                                                 url  
0  http://www.vuelax.com/2018/01/21/cdmx-a-india-...  
1  http://www.vuelax.com/2018/01/20/gdl-a-la-paz-...  
2  http://www.vuelax.com/2018/01/20/cdmx-a-india-...  
3  http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-...  
4  http://www.vuelax.com/2018/01/19/cun-a-amsterd...  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 0 to 41
Data columns (total 6 columns):
date           1350 non-null object
destination    1350 non-null object
note           11 non-null object
origin         1350 non-null object
price          1350 non-null object
url            1350 non-null object
dtypes: object(

In [16]:
clean = pd.read_csv(join(base_dir, "clean.csv"), index_col = 0)

strip_blanks = lambda x: x.strip()
strip_dot = lambda x: x.strip('.')

print(clean[['origin','destination','price']].tail(30))
clean.origin = clean.origin.apply(strip_blanks)
clean.destination = clean.destination.apply(strip_blanks)
clean.price = clean.price.apply(strip_dot)
print(clean[['origin','destination','price']].tail(30))


clean.to_csv(join(base_dir, "clean.csv"), encoding='utf-8')

                         origin                       destination    price
12                   CDMX y GDL             Chihuahua y viceversa    1,814
13                         CDMX  El Calafate, Patagonia Argentina   10,829
14       CDMX y 23 ciudades más                 San Francisco, CA    3,795
15       CDMX y 23 ciudades más                   Toronto, Canadá   8,486.
16                         CDMX                 Santa Clara, Cuba    4,666
17       CDMX y 23 ciudades más                  Montreal, Canadá   8,367.
18                         CDMX                            Madrid  11,866.
19                         CDMX                         Barcelona  11,921.
20         CDMX, MTY, GDL y CUN                  El Cairo, Egipto  10,038.
21                         CDMX             San Juan, Puerto Rico    4,292
22                   CDMX y GDL         San Francisco, California    3,970
23                         CDMX                    Beirut, Líbano  13,219.
24  CDMX, GDL y 22 ciudad

In [17]:
clean = pd.read_csv(join(base_dir, "clean.csv"), index_col = 0, encoding='utf-8')
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 0 to 41
Data columns (total 6 columns):
date           1350 non-null object
destination    1350 non-null object
note           11 non-null object
origin         1350 non-null object
price          1350 non-null object
url            1350 non-null object
dtypes: object(6)
memory usage: 73.8+ KB


In [18]:
convert_price = lambda price: float(price.replace(',',''))
clean.price = clean['price'].apply(convert_price)
clean.to_csv(join(base_dir, "clean.csv"), encoding='utf-8')

In [19]:
clean = pd.read_csv(join(base_dir, "clean.csv"), index_col = 0, parse_dates=['date'], encoding='utf-8')
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 0 to 41
Data columns (total 6 columns):
date           1350 non-null datetime64[ns]
destination    1350 non-null object
note           11 non-null object
origin         1350 non-null object
price          1350 non-null float64
url            1350 non-null object
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 73.8+ KB


In [28]:
import nltk
tolist  = lambda origin: [t.strip() for t in nltk.tokenize.regexp.regexp_tokenize(origin, r'[y,\.\?!"]\s*', gaps=True)]

separate_origins = []

for index, row in clean.iterrows():
    origins = tolist(row['origin'])
    for origin in origins:
        separate_origins.append([origin, row['destination'],
                                row['date'], row['price'],
                                row['note'], row['url']])
        
separa_origin_df = pd.DataFrame(separate_origins, columns=['origin', 'destination',
                                                           'date', 'price', 
                                                           'note', 'url'])

separa_origin_df.to_csv(join(base_dir, "separate_origins.csv"), encoding='utf-8')

In [53]:
import geocoder

locations = sorted(set(list(separa_origin_df.origin.unique()) + list(separa_origin_df.destination.unique())))

real_locations = {'CDMX': 'Ciudad de México', 
                  'CUN': 'Cancún', 'GDL': 'Guadalajara',
                  'L.A.': 'Los Angeles', 'LA': 'Los Angeles',
                  'MTY': 'Monterrey', 'NYC': 'New York City', 
                  'PUE': 'Puebla', 'QRO': 'Querétaro',
                  'SLP': 'San Luis Potosí',
                  'TIJ': 'Tijuana', 'VER': 'Veracruz'
                 }

location_dic = {}
for l in locations[21:]:
    l = real_locations.get(l, l)
    g = geocoder.google(l,key = 'GOOGLE KEY API')
    if g.ok:
        location_dic[l] = g.latlng

In [61]:
get_real_name = lambda l: real_locations.get(l, l)
get_lat = lambda l: location_dic.get(l, (np.nan,np.nan))[0]
get_long = lambda l: location_dic.get(l, (np.nan,np.nan))[1]

separa_origin_df['origin'] = separa_origin_df.origin.apply(get_real_name)
separa_origin_df['origin_lat'] = separa_origin_df.origin.apply(get_lat)
separa_origin_df['origin_long'] = separa_origin_df.origin.apply(get_long)
separa_origin_df['destination_lat'] = separa_origin_df.destination.apply(get_lat)
separa_origin_df['destination_long'] = separa_origin_df.destination.apply(get_long)

separa_origin_df = separa_origin_df[['origin','origin_lat','origin_long',
                                     'destination','destination_lat','destination_long',
                                     'date', 'price', 'url','note']]
separa_origin_df.to_csv(join(base_dir, "separate_origins.csv"), encoding='utf-8')
separa_origin_df.head()

,origin,origin_lat,origin_long,destination,destination_lat,destination_long,date,price,url,note
0,Ciudad de México,19.432608,-99.133208,India,20.593684,78.962880,2018-01-21,8621.0,http://www.vuelax.com/2018/01/21/cdmx-a-india-...,NaN
1,Guadalajara,20.659699,-103.349609,La Paz,-16.489689,-68.119294,2018-01-20,1195.0,http://www.vuelax.com/2018/01/20/gdl-a-la-paz-...,NaN
2,Ciudad de México,19.432608,-99.133208,India,20.593684,78.962880,2018-01-20,8621.0,http://www.vuelax.com/2018/01/20/cdmx-a-india-...,NaN
3,Guadalajara,20.659699,-103.349609,China,35.861660,104.195397,2018-01-19,11046.0,http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-...,NaN
4,Ciudad de México,19.432608,-99.133208,China,35.861660,104.195397,2018-01-19,11046.0,http://www.vuelax.com/2018/01/19/gdl-y-cdmx-a-...,NaN
